# Web scraping Linkedin
## DS Market research

### Importar Pandas y activar el Chrome Driver

In [ ]:
import pandas as pd

from selenium import webdriver

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import time

In [91]:
chrome_driver = '/Users/cris/Documents/Repositorios/local_docs/chromedriver'

options = webdriver.ChromeOptions() # Añadimos un dic con todas las opciones de configuración de Chrome.

# options.add_argument('headless')

# Headless hace que el web scrapping vaya viendo lo que hace o no.

# Podemos configurar diversas opciones en líneas sucesivas.

driver = webdriver.Chrome(executable_path = chrome_driver, options = options)

In [92]:
def login(username, password):
    driver.get('https://www.linkedin.com/uas/login')

    username_input = driver.find_element_by_id('username')
    username_input.send_keys(username)

    password_input = driver.find_element_by_id('password')
    password_input.send_keys(password)
    password_input.submit()

In [93]:
username = 'c.espanaces@gmail.com'
password = 'aay}1\jsL<}L9qy;w}br'

login(username, password)

#### Iniciar búsqueda

In [30]:
search_job = driver.find_elements_by_tag_name('input')

In [31]:
search_job[1].send_keys('Data analyst')

In [32]:
search_job[1].send_keys(Keys.ENTER)

#### Pinchar en empleo:

In [ ]:
driver.back()

In [33]:
b_jobs = driver.find_elements_by_tag_name('button')

In [34]:
b_jobs[8].click()

#### Recoger info puesto trabajo:

In [ ]:
driver.back()

In [ ]:
content = driver.find_elements_by_tag_name('div.flex-grow-1.artdeco-entity-lockup__content.ember-view')

In [ ]:
lista_content = []

for i in content:
    lista_content.append(i.find_element_by_tag_name('a').get_attribute('href'))

## Método 1

#### Buscar información dentro de cada página

In [ ]:
# Creo lista con todos la información recopilada de la empresa, por orden.

def info_collect1(driver):

    info_df = []

    title = driver.find_elements_by_tag_name('div.jobs-details-top-card__content-container')

    title = title[0].text.splitlines()
    
    if len(title) < 9:
        info_df = []
        
        title = driver.find_elements_by_tag_name('div.jobs-details-top-card__content-container')

        title = title[0].text.splitlines()
    
    elif title[5] == 'Nuevo':
        title = [title[2], title[0], title[4], title[7], title[9]]
    
    else:
        title = [title[2], title[0], title[4], title[6], title[8]]
        
    
    info_job_comp = driver.find_elements_by_tag_name('li.artdeco-list__item.jobs-details-job-summary-list__item.jobs-details-job-summary-list__item--bullet')
    
    info_job_comp = info_job_comp[0].text.splitlines()
    
    
    job_desc = driver.find_elements_by_tag_name('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal')
    
    job_desc1 = job_desc[0].text.splitlines()[:-8]    
    
    tuple(job_desc)
    
    job_desc2 = job_desc[0].text.splitlines()[-8:]
    
    job_desc2 = [job_desc2[1], job_desc2[3], job_desc2[5], job_desc2[7]]
    
    info_df = title + job_desc2
    
    return info_df

In [ ]:
# Creo los títulos de las columnas del df.

titulos_col = ['Nombre empresa', 'Puesto ofertado', 'Ubicación', 'Días activos', 'Nº Visualizaciones', 'Nivel experiencia', 'Sector', 'Tipo jornada', 'Funciones laborales']

In [ ]:
jobs_list1 = pd.DataFrame()

In [ ]:
for i in range(len(lista_content)):
    content[i].click()
    info_df = info_collect1(driver)
    df_prueba1 = pd.DataFrame([info_df], columns = titulos_col)
    jobs_list1 = jobs_list1.append(df_prueba1)

## Método 2

In [ ]:
# Creo lista con todos la información recopilada de la empresa, por orden.

def info_collect2(driver):

    info_df = []
    
    time.sleep(1)
    
    try:
        boton1 = driver.find_element_by_id('ember106')
        boton1.click()
    except:
        None
    
    try:
        empresa = driver.find_element_by_tag_name('a.ember-view.t-black.t-normal').text
    except:
        empresa = driver.find_element_by_tag_name('span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black').text
    
    puesto = driver.find_element_by_tag_name('h1.t-24.t-bold').text
    ubicacion = driver.find_element_by_tag_name('span.jobs-unified-top-card__bullet').text
    tiempo_puesto = driver.find_element_by_tag_name('span.jobs-unified-top-card__posted-date').text
    
    try:
        solicitud = driver.find_element_by_tag_name('span.jobs-unified-top-card__applicant-count').text
    except:
        solicitud = ''
    
    info_pr = [empresa, puesto, ubicacion, tiempo_puesto, solicitud]
    
    try:
        boton2 = driver.find_element_by_id('ember60')
        boton2.click()
    except:
        None
    
    try:
        info_job_comp = driver.find_element_by_class_name('jobs-description-details').text
        info_job_comp = info_job_comp.splitlines()
        if info_job_comp[1]=='Nivel de experiencia':
            info_job_comp = [info_job_comp[2], info_job_comp[4], info_job_comp[6], info_job_comp[8]]
        
        elif info_job_comp[0]=='Nivel de experiencia':
            info_job_comp = [info_job_comp[1], info_job_comp[3], info_job_comp[5], info_job_comp[7]]
        
        else:
            info_job_comp = ['', info_job_comp[3], info_job_comp[5], info_job_comp[7]]
            
    except:
        info_job_comp = driver.find_element_by_class_name('jobs-description-details').text
        info_job_comp = info_job_comp.splitlines()
        info_job_comp = [info_job_comp[1], info_job_comp[3], '', '']
    
    description = driver.find_elements_by_tag_name('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal')
    description = description[0].text
    
    info_df = info_pr + info_job_comp

    return info_df

In [ ]:
# Creo los títulos de las columnas del df.

titulos_col = ['Nombre empresa', 'Puesto ofertado', 'Ubicación', 'Días activos', 'Nº Solicitudes', 'Nivel experiencia', 'Sector', 'Tipo jornada', 'Funciones laborales']

In [ ]:
jobs_list = pd.DataFrame()

In [ ]:
def contenido(driver):
    parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")

    elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

    x = []
    for i in elementList:
        x.append(i.find_element_by_tag_name('a').get_attribute('href'))
    
    return x

In [ ]:
for i in contenido(driver):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(i)
    info_df = info_collect2(driver)
    df_prueba1 = pd.DataFrame([info_df], columns = titulos_col)
    jobs_list = jobs_list.append(df_prueba1)
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    

In [ ]:
driver.find_element_by_class_name('jobs-description-details').text

In [ ]:
jobs_list

In [ ]:
driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[-1])
driver.get(lista_content[20])

In [ ]:
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
driver.close()

In [ ]:
jobs_list

In [ ]:
parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")


In [ ]:
def contenido(driver):
    parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")

    elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

    x = []
    for i in elementList:
        x.append(i.find_element_by_tag_name('a').get_attribute('href'))
    
    return x

In [ ]:
x = []
for i in elementList:
    x.append(i.find_element_by_tag_name('a').get_attribute('href'))

In [ ]:
len

In [ ]:
len(x)

In [ ]:
count = 1
number = driver.find_elements_by_tag_name('li.artdeco-pagination__indicator.artdeco-pagination__indicator--number.ember-view')
count += 1


In [ ]:
for i in number:
    print(i.text)

## Método 3

In [ ]:
parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")

elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

x = []
for i in elementList:
    x.append(i.find_element_by_tag_name('a').get_attribute('href'))

In [ ]:
# Creo lista con todos la información recopilada de la empresa, por orden.

def info_collect(driver):

    info_df = []
    
    title = driver.find_elements_by_tag_name('div.jobs-details-top-card__content-container')
   
    title = title[0].text.splitlines()
    
    
    if len(title) < 9:

        title = [title [0], '', '', '', '']

    elif title[5] == 'Nuevo':
        title = [title[2], title[0], title[4], title[7], title[9]]

    else:
        title = [title[2], title[0], title[4], title[6], title[8]]
        
        
    
    info_job_comp = driver.find_elements_by_tag_name('li.artdeco-list__item.jobs-details-job-summary-list__item.jobs-details-job-summary-list__item--bullet')
    
    info_job_comp = info_job_comp[0].text.splitlines()
    
    
    job_desc = driver.find_elements_by_tag_name('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal')
    
    job_desc1 = job_desc[0].text.splitlines()[:-8]    
    
    tuple(job_desc)
    
    job_desc2 = job_desc[0].text.splitlines()[-8:]
    
    job_desc2 = [job_desc2[1], job_desc2[3], job_desc2[5], job_desc2[7]]
    
    info_df = title + job_desc2
    
    return info_df

In [ ]:
def charge_web(driver):
    parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")
    elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

    for i in range(len(elementList)):    
        elementList[i].click()
        time.sleep(1)
        parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")
        elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")
    
    return elementList

In [ ]:
# Creo los títulos de las columnas del df.

titulos_col = ['Nombre empresa', 'Puesto ofertado', 'Ubicación', 'Días activos', 'Nº Solicitudes', 'Nivel experiencia', 'Sector', 'Tipo jornada', 'Funciones laborales']

In [ ]:
jobs_list = pd.DataFrame()

In [ ]:
def change_page(driver, count):
    number = driver.find_elements_by_tag_name('li.artdeco-pagination__indicator.artdeco-pagination__indicator--number.ember-view')

    lista_str = []
    lista_num = []

    for i in range(1, 34):
        lista_str.append(str(i))
        lista_num.append(i)

    for i in lista_num:

        if number[i].text == lista_str[i]:
            number[i].click()
            
    else:
        break

In [ ]:
count = 1
while count < 2:
    elementList = charge_web(driver)
    
    for i in range(len(elementList)):
        elementList[i].click()
        time.sleep(1)
        info_df = info_collect(driver)
        df_prueba = pd.DataFrame([info_df], columns = titulos_col)
        jobs_list = jobs_list.append(df_prueba)
        time.sleep(5)

In [ ]:
number = driver.find_elements_by_tag_name('li.artdeco-pagination__indicator.artdeco-pagination__indicator--number.ember-view')

lista_str = []
lista_num = []

for i in range(1, 34):
    lista_str.append(str(i))
    lista_num.append(i)

for i in lista_num:

    if number[i].text == lista_str[i]:
        number[i].click()

In [ ]:
parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")
elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

for i in range(len(elementList)):    
    elementList[i].click()
    time.sleep(1)
    parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")
    elementList = parentElement.find_elements_by_clas

## Método 4

In [94]:
# Creo lista con todos la información recopilada de la empresa, por orden.

def info_collect(driver):

    info_df = []
    
    title = driver.find_elements_by_tag_name('div.jobs-details-top-card__content-container')
   
    title = title[0].text.splitlines()
    
    
    if len(title) < 9:

        title = [title [0], '', '', '', '']

    elif title[5] == 'Nuevo':
        title = [title[2], title[0], title[4], title[7], title[9]]

    else:
        title = [title[2], title[0], title[4], title[6], title[8]]
        
        
    
    info_job_comp = driver.find_elements_by_tag_name('li.artdeco-list__item.jobs-details-job-summary-list__item.jobs-details-job-summary-list__item--bullet')
    
    info_job_comp = info_job_comp[0].text.splitlines()
    
    
    job_desc = driver.find_elements_by_tag_name('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal')
    
    job_desc1 = [job_desc[0].text]
    

    
    job_desc2 = job_desc[0].text.splitlines()[-8:]
    
    job_desc2 = [job_desc2[1], job_desc2[3], job_desc2[5], job_desc2[7]]
    
    info_df = title + job_desc2 + job_desc1
    
    return info_df

In [95]:
# Creo los títulos de las columnas del df.

titulos_col = ['Nombre empresa', 'Puesto ofertado', 'Ubicación', 'Días activos', 'Nº Solicitudes', 'Nivel experiencia', 'Sector', 'Tipo jornada', 'Funciones laborales', 'Descripción']

In [96]:
jobs_list = pd.DataFrame()

In [163]:
parentElement = driver.find_element_by_class_name("jobs-search-results__list.list-style-none")
elementList = parentElement.find_elements_by_class_name("mr1.artdeco-entity-lockup__image.artdeco-entity-lockup__image--type-square.ember-view")

In [164]:
for i in range(len(elementList)):
    elementList[i].click()
    time.sleep(2)
    info_df = info_collect(driver)
    df_prueba = pd.DataFrame([info_df], columns = titulos_col)
    jobs_list = jobs_list.append(df_prueba)

In [169]:
jobs_list

,Nombre empresa,Puesto ofertado,Ubicación,Días activos,Nº Solicitudes,Nivel experiencia,Sector,Tipo jornada,Funciones laborales,Descripción
0,Amazon,Junior Data Analyst,"Barcelona, Cataluña, España",Publicado hace hace 6 horas,66 visualizaciones,Sin experiencia,Software Servicios y tecnologías de la informa...,Jornada completa,Estrategia/planificación Análisis Tecnología d...,Description\n\nJunior Data Analyst\nWe are hir...
0,Packlink.com,Business Data Analyst,"Madrid, Comunidad de Madrid, España",Publicado hace hace 1 semana,638 visualizaciones,Intermedio,Servicios y tecnologías de la información Logí...,Jornada completa,Negocios Tecnología de la información Análisis,Do you want to be a part of a fast-paced compa...
0,Mutua Madrileña,Data Analyst,"Madrid, Comunidad de Madrid, España",Publicado hace hace 2 semanas,3490 visualizaciones,Algo de responsabilidad,Seguros,Jornada completa,Otro,Anunciado por\nAdriana Sánchez\nExperta Desarr...
0,Lookiero,Data Analyst,"Respaldiza, País Vasco / Euskadi, España",Publicado hace hace 2 semanas,3374 visualizaciones,Intermedio,Sector textil,Jornada completa,Ingeniería Tecnología de la información,Anunciado por\nElena Martín\nTalent Acquisitio...
0,YEGO,Junior Data Analyst,"Barcelona, Cataluña, España",Publicado hace hace 3 días,1122 visualizaciones,Sin experiencia,Software Marketing y publicidad Transporte por...,Jornada completa,Ciencias Análisis Tecnología de la información,Mission\nAs a Junior Data Analyst you will sup...
...,...,...,...,...,...,...,...,...,...,...
0,Toronto Catholic District School Board,Senior Database Analyst,"Valencia/València, Comunidad Valenciana / Comu...",Publicado hace hace 1 mes,17 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Tecnología de la información,APPLICATIONS ARE INVITED FOR THE POSITION OF S...
0,Toronto Catholic District School Board,Senior Database Analyst,"Tomares, Andalucía, España",Publicado hace hace 1 mes,21 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Tecnología de la información,APPLICATIONS ARE INVITED FOR THE POSITION OF S...
0,Solera,FP&A Senior Analyst,"Comunidad de Madrid, España",Publicado hace hace 3 semanas,34 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Finanzas Ventas,Who We Are\n\nSolera is a global leader in dat...
0,Amazon,HR Program Manager – Analytics and Reporting,"Córdoba y alrededores, España",Publicado hace hace 6 días,17 visualizaciones,Intermedio,Servicios y tecnologías de la información Soft...,Jornada completa,Gestión de proyectos Tecnología de la información,EU COUNTRY LOCATION FLEXIBLE***Amazon is a com...


In [167]:
jobs_list.to_csv(r'/Users/cris/Documents/Repositorios/EDA_DS_Market/info_data_analysis.csv', index = False)


In [168]:
jobs_list

,Nombre empresa,Puesto ofertado,Ubicación,Días activos,Nº Solicitudes,Nivel experiencia,Sector,Tipo jornada,Funciones laborales,Descripción
0,Amazon,Junior Data Analyst,"Barcelona, Cataluña, España",Publicado hace hace 6 horas,66 visualizaciones,Sin experiencia,Software Servicios y tecnologías de la informa...,Jornada completa,Estrategia/planificación Análisis Tecnología d...,Description\n\nJunior Data Analyst\nWe are hir...
0,Packlink.com,Business Data Analyst,"Madrid, Comunidad de Madrid, España",Publicado hace hace 1 semana,638 visualizaciones,Intermedio,Servicios y tecnologías de la información Logí...,Jornada completa,Negocios Tecnología de la información Análisis,Do you want to be a part of a fast-paced compa...
0,Mutua Madrileña,Data Analyst,"Madrid, Comunidad de Madrid, España",Publicado hace hace 2 semanas,3490 visualizaciones,Algo de responsabilidad,Seguros,Jornada completa,Otro,Anunciado por\nAdriana Sánchez\nExperta Desarr...
0,Lookiero,Data Analyst,"Respaldiza, País Vasco / Euskadi, España",Publicado hace hace 2 semanas,3374 visualizaciones,Intermedio,Sector textil,Jornada completa,Ingeniería Tecnología de la información,Anunciado por\nElena Martín\nTalent Acquisitio...
0,YEGO,Junior Data Analyst,"Barcelona, Cataluña, España",Publicado hace hace 3 días,1122 visualizaciones,Sin experiencia,Software Marketing y publicidad Transporte por...,Jornada completa,Ciencias Análisis Tecnología de la información,Mission\nAs a Junior Data Analyst you will sup...
...,...,...,...,...,...,...,...,...,...,...
0,Toronto Catholic District School Board,Senior Database Analyst,"Valencia/València, Comunidad Valenciana / Comu...",Publicado hace hace 1 mes,17 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Tecnología de la información,APPLICATIONS ARE INVITED FOR THE POSITION OF S...
0,Toronto Catholic District School Board,Senior Database Analyst,"Tomares, Andalucía, España",Publicado hace hace 1 mes,21 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Tecnología de la información,APPLICATIONS ARE INVITED FOR THE POSITION OF S...
0,Solera,FP&A Senior Analyst,"Comunidad de Madrid, España",Publicado hace hace 3 semanas,34 visualizaciones,Algo de responsabilidad,Servicios y tecnologías de la información Soft...,Jornada completa,Finanzas Ventas,Who We Are\n\nSolera is a global leader in dat...
0,Amazon,HR Program Manager – Analytics and Reporting,"Córdoba y alrededores, España",Publicado hace hace 6 días,17 visualizaciones,Intermedio,Servicios y tecnologías de la información Soft...,Jornada completa,Gestión de proyectos Tecnología de la información,EU COUNTRY LOCATION FLEXIBLE***Amazon is a com...
